word based text generation using LSTM model

the training texts are samples from the following texts --

https://www.tibethouse.jp/about/buddhism/text/pdfs/Bodhisattvas_way_English.pdf http://promienie.net/images/dharma/books/shantideva_way-of-bodhisattva.pdf

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [ ]:
path = "/content/drive/My Drive/test.txt"

In [ ]:
#importing all required libraries 
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [ ]:
#reading the file
with open(path, 'r' , encoding = "utf-8") as file:  
  filer = file.read()

In [ ]:
import re
filer = re.sub("\d+", "", filer) #to remove the digits/numbers from text

In [ ]:
filer[0:100]

'. All that we are is the result of what we have thought: it is founded on our thoughts, it is made u'

creating tokens and word , value dict of vocabulory 

In [ ]:
tokenizer = Tokenizer() #creates a instance of Tokenizer

corpus = filer.lower().split("\n") # creates a list of strings seperated by "\n."

tokenizer.fit_on_texts(corpus) #tkenizer creates tokens on the data in corpus
word_index = tokenizer.word_index # return dict of key as words and value being their the rank of occurence  
total_words = len(tokenizer.word_index) +1

print(word_index)
print(total_words)

{'the': 1, 'of': 2, 'and': 3, 'i': 4, 'to': 5, 'is': 6, 'a': 7, 'in': 8, 'not': 9, 'who': 10, 'for': 11, 'be': 12, 'all': 13, 'this': 14, 'by': 15, 'it': 16, 'my': 17, 'that': 18, 'with': 19, 'will': 20, 'if': 21, 'he': 22, 'as': 23, 'are': 24, 'should': 25, 'have': 26, 'from': 27, 'do': 28, 'they': 29, 'but': 30, 'no': 31, 'mind': 32, 'shall': 33, 'me': 34, 'what': 35, 'like': 36, 'when': 37, 'his': 38, 'man': 39, 'those': 40, 'has': 41, 'others': 42, 'their': 43, 'even': 44, 'there': 45, 'so': 46, 'an': 47, 'him': 48, 'then': 49, 'body': 50, 'or': 51, 'beings': 52, 'world': 53, 'myself': 54, 'evil': 55, 'why': 56, 'such': 57, 'them': 58, 'one': 59, 'good': 60, 'may': 61, 'am': 62, 'way': 63, 'on': 64, 'having': 65, 'you': 66, 'these': 67, 'life': 68, 'at': 69, 'being': 70, 'other': 71, 'does': 72, 'can': 73, 'let': 74, 'which': 75, 'great': 76, 'pain': 77, 'well': 78, 'through': 79, 'wise': 80, 'people': 81, 'happiness': 82, 'never': 83, 'therefore': 84, 'upon': 85, 'now': 86, 'free'

creating the training data

In [ ]:
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0] # 
	for i in range(0, len(token_list) - 5 , 1):
		sequence_slices = token_list[i:i+6] #sequences of length starting from 2 to end of sentence
		input_sequences.append(sequence_slices)

# pad sequences 
max_sequence_len = 6
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')) #padding seuences apecifying the maximum length of sequence and the type of padding 
#pre padding starts padding 0s from start 
# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1] #xs are the input sequence without the last eement and last element for each sequence is its corresponding label


In [ ]:
import re
import numpy as np
def random_shuffle(x, y):
    permutation = np.random.permutation(x.shape[0])
    shuffled_x = x[permutation]
    shuffled_y = y[permutation]
    return shuffled_x, shuffled_y
 
#to build a one hot encoded matrix of shape (# of seqs), (length of seq), (num of possible character    
x_matrix = []
for i in range(len(xs)):    
    one_hot_chars = np.eye(total_words)[xs[i]]
    x_matrix.append(one_hot_chars)
x_matrix = np.array(x_matrix)

# build the one hot encoded matrix of shape (# of seqs), (num of possible characters)
y_matrix = np.eye(total_words)[labels]

# shuffle the data
x_matrix, y_matrix = random_shuffle(x_matrix, y_matrix)

# Save the data to a numpy archive format
np.savez(r'C:\Users\hp\Desktop\internship\speech_recognition\data.npz', x=x_matrix, y=y_matrix)

Model arcitecture 
build and train the model

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers.core import *
from keras.layers import Input, Bidirectional, LSTM, Multiply

def build_model():
    x = Input((max_sequence_len-1, total_words))
    
    encoder = LSTM(400, return_sequences=True, recurrent_dropout=.35, dropout=.3)(x)
    attention = Dense(1, activation='tanh')(encoder)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(400)(attention)
    attention = Permute([2,1])(attention)
    attention = Multiply()([encoder, attention])
    
    decoder = LSTM(400, recurrent_dropout=.35, dropout=.3)(attention)
    y = Dense(total_words, activation='softmax')(decoder)
    model = Model(inputs=x, outputs=y)
    return model

In [ ]:
key_list = list(word_index.keys()) 
val_list = list(word_index.values())

In [ ]:
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy
from keras.callbacks import TensorBoard, ModelCheckpoint
import numpy as np
from keras.utils import plot_model


BATCH_SIZE = 128
EPOCHS = 200


data = np.load(r'C:\Users\hp\Desktop\internship\speech_recognition\data.npz')
x_seqs, y_chars = data['x'], data['y']
assert x_seqs.shape[0] == y_chars.shape[0]

model = build_model()
model.compile(optimizer=Adam(.001), loss='categorical_crossentropy', metrics=[categorical_accuracy, 'acc'])
model.summary()

history = model.fit(x_seqs, y_chars, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5, 4231)]    0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 5, 400)       7411200     input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 5, 1)         401         lstm[0][0]                       
__________________________________________________________________________________________________
flatten (Flatten)               (None, 5)            0           dense[0][0]                      
_______________________________________________________________________________________

In [95]:
#test string
test_str = "A good way to make sure you don’t miss anything important at a meeting is to record it so you can listen to it again later."

In [126]:
import re
test_str = re.sub("\d+", "", test_str) #to remove the digits/numbers from text

creating test sequences

In [127]:
test_sequences = []

token_list = tokenizer.texts_to_sequences([test_str])[0] # 
for i in range(len(token_list)-5):
		sequence_slices = token_list[i:i+5] #sequences of length 5
		test_sequences.append(sequence_slices)


In [128]:
test_x_matrix = []
for i in range(len(test_sequences)):
    one_hot_chars = np.eye(total_words)[test_sequences[i]]
    test_x_matrix.append(one_hot_chars)
test_x_matrix = np.array(test_x_matrix)
assert test_x_matrix.shape == (len(test_sequences), max_sequence_len-1, total_words), print(test_x_matrix.shape)

Generating text for a given test string


In [129]:
ccc = model.predict(test_x_matrix)

In [130]:
strin = ""
for i in range(len(ccc)):
    ind = np.argmax(ccc[i])
    strin += (key_list[val_list.index(ind)] + " ")
    
print(strin)

himself who art with start death brahmana who start occur it give with little 
